# NIH Common Data Elements Lookup Program
J. Michael Dean, MD
July 31, 2024

This notebook is an exploration of loading all the CDEs from the NIH
and creating a RAG application that will allow DCC staff to easily lookup
CDEs that are relevant to data items in one of our protocols.

In [1]:
# Install necessary modules
# !pip install -qU langchain langchain_community
# !pip install -qU pypdf
# !pip install yt_dlp
# !pip install pydub
# !pip install -qU chromadb
# !pip install -qU langchain_openai
# !pip install -qU sentence_transformers

In [2]:
# Load dependencies
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_community.document_loaders import JSONLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.text_splitter import RecursiveJsonSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from openai import OpenAI
from IPython.display import Markdown, display

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import shutil

In [4]:
# Let's load the CSV file of CDE codes
from langchain_community.document_loaders import CSVLoader
# file_path = './SearchExport.csv'
file_path = './output.csv'
# loader = CSVLoader(file_path='./SearchExport.csv',
#                   source_column='Name',
#                   )
loader = CSVLoader(file_path=file_path)

In [5]:
docs = loader.load()
print(docs[5].page_content[:1000])
print(docs[5].metadata)
print(len(docs))

Data Element Name: Address State Code
Question Text: State
Definition: The state for the address to describe where a mail piece is intended to be delivered.
Value Type: Value List
Permissible Values: AL; AK; AZ; AR; CA; CO; CT; DC; DE; FL; GA; HI; ID; IL; IN; IA; KS; KY; LA; ME
Related Concepts: Address State Code
Registration Status: Qualified
Identifier: 
Reference URL: https://cde.nlm.nih.gov/deView?tinyId=6AT_JFxD1
{'source': './output.csv', 'row': 5}
24328


In [6]:
from tqdm.autonotebook import tqdm, trange
# Embed the documents and store in Chromadb
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True}
embedding_function = HuggingFaceBgeEmbeddings(model_name=model_name,
                                              encode_kwargs=encode_kwargs)
persist_directory = 'docs/chroma/'
# if os.path.exists("docs/chroma/") and os.path.isdir(persist_directory):
#     shutil.rmtree("docs/")
#     print(f"The directory {persist_directory} has been deleted.")
# else:
#     print(f"The directory {persist_directory} did not exist.")
# persist_directory = 'docs/chroma/'
vectordb = Chroma.from_documents(documents=docs,
                                 embedding=embedding_function,
                                 persist_directory=persist_directory)


/var/folders/5d/11s8m5v96vq615m0c6399q8r0000gn/T/ipykernel_45977/1773680942.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [17]:
llm = OpenAI()

In [18]:
# Create a prompt for our RAG application

In [19]:
def getRagData(query:str, k=5):
    rag_return = vectordb.similarity_search(query, k=k)
    context = """
    You should answer the user query if the information is contained in this context.  
    Otherwise you should say that you do not know.
    """
    for r in rag_return:
        context = context + r.page_content
    prompt = context + "\n" + query
    # print(prompt)
    return prompt

In [26]:
def askQuestion(query: str, k=5):
    prompt = getRagData(query,k)
    # prompt = query
    completion = llm.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[  
            {"role": "system", "content": """
            You are an expert helpful assistant who looks up Common Data Elements (CDEs) that can be used in clinical research.  
            Only return CDEs that have a Registration Status of Qualified.  
            
            Use the following format to return each CDE:
          
                 Data Element Name:\n
                 Question Text \n
                 Definition \n
                 Value Type \n
                 Permissible Values (if there aren't any, leave this off)\n
                    Value 1 \n
                    Value 2 \n
                    etc.
                 Related Concepts (if there aren't any, leave this off) \n
                 Registration Status \n
                 Identifier (if there aren't any, leave this off)\n
                 Reference URL (Use data element name for the link)
                 ---
                 
            """
            },   
            {"role": "user", "content": prompt}
        ], 
        temperature=0.01
    )
    display(Markdown((completion.choices[0].message.content)))

In [27]:
    # fieldnames = ['Data Element Name', 'Question Text', 'Definition', 'Value Type', 'Permissible Values', 'Related Concepts',
                 # 'Registration Status', 'Identifier']

In [28]:
askQuestion("Age of a child subject at time of enrollment in a study",25)

Data Element Name: Age in years

Question Text: Subject's age (recorded in years):

Definition: Value for participant's subject age, calculated as elapsed time since the birth of the participant/subject in years. The subjects age is typically recorded to the nearest full year completed, e.g. 11 years and 6 months should be recorded as 11 years.; Value for participant's subject age recorded in years.

Value Type: Number

Registration Status: Qualified

Identifier: BRICS Variable Name: AgeYrs

Reference URL: [Age in years](https://cde.nlm.nih.gov/deView?tinyId=B7YQnL_vF)

---

Data Element Name: Age (years) at Study Enrollment

Question Text: Age at Study Entry (years)

Definition: The fractional age in years of the participant at study entry. For example, if a participant is 23 years and 3 months old at study entry, code 23.25.

Value Type: Number

Registration Status: Qualified

Identifier: ISTH: 4:3:10:438

Reference URL: [Age (years) at Study Enrollment](https://cde.nlm.nih.gov/deView?tinyId=PBpuM_I_Av)

---

Data Element Name: Age (months) at Study Enrollment

Question Text: Age at Study Entry (months)

Definition: The age in months of the participant at study entry.

Value Type: Number

Registration Status: Qualified

Identifier: ISTH: 4:3:10:673

Reference URL: [Age (months) at Study Enrollment](https://cde.nlm.nih.gov/deView?tinyId=N8VI8hlfOI)

---

Data Element Name: Grade Level

Question Text: If the participant is less than 18 years old, what is the highest grade the participant has started?

Definition: For participants receiving special educational services, please select the grade level assigned by their education provider.; No schooling

Value Type: Value List

                    Value 1: No schooling 
                    
                    Value 2: Preschool or nursery school 
                    
                    Value 3: Kindergarten 
                    
                    Value 4: 1st Grade 
                    
                    Value 5: 2nd Grade 
                    
                    Value 6: 3rd Grade 
                    
                    Value 7: 4th Grade 
                    
                    Value 8: 5th Grade 
                    
                    Value 9: 6th Grade 
                    
                    Value 10: 7th Grade 
                    
                    Value 11: 8th Grade 
                    
                    Value 12: 9th Grade 
                    
                    Value 13: 10th Grade 
                    
                    Value 14: 11th Grade 
                    
                    Value 15: 12th Grade 
                    
                    Value 16: GED 
                    
                    Value 17: College 
                    
                    Value 18: Don’t Know 
                    
                    Value 19: N/A - Participant is 18 years or older 

Registration Status: Qualified

Identifier: GRDR: GRDR020

Reference URL: [Grade Level](https://cde.nlm.nih.gov/deView?tinyId=F2kfwY2evhv)

---

Data Element Name: Children under 18 years age count

Question Text: Children under 18 years age count

Definition: Count of the number of children the participant/subject has under 18 years of age

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C58327

Reference URL: [Children under 18 years age count](https://cde.nlm.nih.gov/deView?tinyId=my_hsIKp6I)

---

Data Element Name: Age of Symptom Onset

Question Text: What was the participant's age in years when they first began experiencing symptoms of the rare disease?

Definition: Prenatal
At birth
Less than 1 year
Age 1 - 89 years
90+ years
Participant never experienced any symptoms
Age of onset unknown

Value Type: Value List

                    Value 1: Prenatal 
                    
                    Value 2: At birth 
                    
                    Value 3: Less than 1 year 
                    
                    Value 4: Age 1 - 89 years 
                    
                    Value 5: 90+ years 
                    
                    Value 6: Participant never experienced any symptoms 
                    
                    Value 7: Age of onset unknown 

Registration Status: Qualified

Identifier: GRDR: GRDR028

Reference URL: [Age of Symptom Onset](https://cde.nlm.nih.gov/deView?tinyId=9Wx7BE0J6V6)

---

Data Element Name: Age first suspected something different range

Question Text: Age first suspected something different range

Definition: Age range of participant/subject when it was first suspected that something might be "different"

Value Type: Value List

                    Value 1: Fetal 
                    
                    Value 2: At birth 
                    
                    Value 3: 1-2 months 
                    
                    Value 4: 3-6 months 
                    
                    Value 5: 7-12 months 
                    
                    Value 6: 13-24 months 
                    
                    Value 7: 25-36 months 
                    
                    Value 8: 37 months - 10 years 
                    
                    Value 9: 11 - 20 years 
                    
                    Value 10: 21-30 years 

Registration Status: Qualified

Identifier: BRICS Variable Name: Age1stSuspectDifferentRng

Reference URL: [Age first suspected something different range](https://cde.nlm.nih.gov/deView?tinyId=5fdUKHfykdu)

---

Data Element Name: Weight measurement pediatric value

Question Text: Weight measurement pediatric value

Definition: Measurement of weight for a participant's/subject's who is < 21 years old.

Value Type: Number

Registration Status: Qualified

Identifier: NINDS caDSR: 2179689

Reference URL: [Weight measurement pediatric value](https://cde.nlm.nih.gov/deView?tinyId=myOuxHm5jrR)

---

Data Element Name: Age at Diagnosis

Question Text: What was the participant's age in years when the formal diagnosis was made?

Definition: Prenatal
At birth
Less than 1 year
Age 1 - 89 years
90+ years
Age of diagnosis unknown

Value Type: Value List

                    Value 1: Prenatal 
                    
                    Value 2: At birth 
                    
                    Value 3: Less than 1 year 
                    
                    Value 4: Age 1 - 89 years 
                    
                    Value 5: 90+ years 
                    
                    Value 6: Age of diagnosis unknown 

Registration Status: Qualified

Identifier: GRDR: GRDR027

Reference URL: [Age at Diagnosis](https://cde.nlm.nih.gov/deView?tinyId=Ys0dsd35Yoy)

---

Data Element Name: What is your child’s current age? Years

Question Text: Age pediatric year part value

Definition: Value of the year part of the pediatric subject/participant's age

Value Type: Number

Registration Status: Qualified

Identifier: BRICS Variable Name: AgePedYrPartVal

Reference URL: [What is your child’s current age? Years](https://cde.nlm.nih.gov/deView?tinyId=6_B2dcf1nM)

---

Data Element Name: What is your child's age?

Question Text: Age below 1 year code

Definition: Code indicating that the subject/participant's age is less than 1 year

Value Type: Value List

                    Value 1: Less than 1 year old 

Registration Status: Qualified

Identifier: BRICS Variable Name: AgeBelow1YearCode

Reference URL: [What is your child's age?](https://cde.nlm.nih.gov/deView?tinyId=V0HdtqU4IG)

---

Data Element Name: Consent

Question Text: If the participant is 18 years or older, was consent given to enter participant's data?

Definition: Yes
No
N/A - Participant is younger than 18 years

Value Type: Externally Defined

Registration Status: Qualified

Reference URL: [Consent](https://cde.nlm.nih.gov/deView?tinyId=7qJXhFmoB7f)

---

Data Element Name: Educational Attainment

Question Text: If the participant is 18 years or older, what is the highest grade or level of school the participant has completed or the highest degree the participant has received?

Definition: Eighth grade or less
More than eighth grade, but did not graduate from high school
Went to a business, trade, or vocational school instead of high school
High school graduate
Completed a GED
Went to a business, trade, or vocational school after high school
Went to college, but did not graduate
Graduated from a college or university
Professional training beyond a four-year college or university
He/she never went to school
He/she went to school, but respondent doesn’t know what level
Respondent doesn’t know if he/she went to school
Refused
Don’t know
N/A - Participant is younger than 18 years

Value Type: Externally Defined

Registration Status: Qualified

Identifier: GRDR: GRDR021

Reference URL: [Educational Attainment](https://cde.nlm.nih.gov/deView?tinyId=kxRtCXGZFkc)

---

Data Element Name: Diagnosis initial age value

Question Text: Diagnosis initial age value

Definition: Age of the participant/subject when initially diagnosed with disease/disorder

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C10501

Reference URL: [Diagnosis initial age value](https://cde.nlm.nih.gov/deView?tinyId=achaOkePd6G)

---

Data Element Name: Age when first injected non-prescription drug

Question Text: Age first non-prescrip drug injec

Definition: The age of the patient when they first injected any kind of non-prescription drug.

Value Type: Number

Registration Status: Qualified

Identifier: LOINC: 91532-2

Reference URL: [Age when first injected non-prescription drug](https://cde.nlm.nih.gov/deView?tinyId=pZiiejNszF)

---

Data Element Name: In the past 7 days My child felt accepted by other kids his/her age.

Question Text: My child felt accepted by other kids his/her age.

Definition: 

Value Type: Value List

                    Value 1: 0 
                    
                    Value 2: 1 
                    
                    Value 3: 2 
                    
                    Value 4: 3 
                    
                    Value 5: 4 

Registration Status: Qualified

Identifier: Assessment Center: Pf3socabil9

Reference URL: [In the past 7 days My child felt accepted by other kids his/her age.](https://cde.nlm.nih.gov/deView?tinyId=Xke9OydhJXx)

---

Data Element Name: Age value

Question Text: Age value

Definition: Value for participant/subject's age, calculated as elapsed time since the birth of the participant/subject.

Value Type: Number

Registration Status: Qualified

Identifier: BRICS Variable Name: AgeVal

Reference URL: [Age value](https://cde.nlm.nih.gov/deView?tinyId=jZT1bnHTnC8)

---

Data Element Name: In the past 7 days My child liked being around other kids his/her age.

Question Text: My child liked being around other kids his/her age.

Definition: 

Value Type: Value List

                    Value 1: 1 
                    
                    Value 2: 2 
                    
                    Value 3: 3 
                    
                    Value 4: 4 
                    
                    Value 5: 5 

Registration Status: Qualified

Identifier: Assessment Center: Pf4socabil4

Reference URL: [In the past 7 days My child liked being around other kids his/her age.](https://cde.nlm.nih.gov/deView?tinyId=7JBqu1uhJ7e)

---

Data Element Name: Age of Onset of Health-related event

Question Text: Age of onset of health-related event Family member

Definition: 

Value Type: Number

Registration Status: Qualified

Identifier: LOINC: 67721-1

Reference URL: [Age of Onset of Health-related event](https://cde.nlm.nih.gov/deView?tinyId=mydxrsqinsx)

---

Data Element Name: Participant Age for Height

Question Text: How old was the participant when this height was measured?

Definition: Select one value for each column from Age for Weight-Height Appendix

Value Type: Externally Defined

Registration Status: Qualified

Identifier: GRDR: GRDR037

Reference URL: [Participant Age for Height](https://cde.nlm.nih.gov/deView?tinyId=XbVXwqST5Hu)

---

Data Element Name: Post conceptional age value

Question Text: Post conceptional age value

Definition: Gestational age plus postnatal age of the participant/subject

Value Type: Number

Registration Status: Qualified

Identifier: NINDS caDSR: 3182713

Reference URL: [Post conceptional age value](https://cde.nlm.nih.gov/deView?tinyId=eYotanGYWFT)

---

Data Element Name: Gestational age child value

Question Text: Gestational age child value

Definition: Time elapsed between the first day of the last normal menstrual period and the day the participant/subject delivered the neonate

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C16056

Reference URL: [Gestational age child value](https://cde.nlm.nih.gov/deView?tinyId=oEsYPyenQpU)

---

Data Element Name: Psychological/neuropsychological previous testing age value

Question Text: Psychological/neuropsychological previous testing age value

Definition: Age at which previous psychological/neuropsychological testing was performed on the participant/subject.

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C14120

Reference URL: [Psychological/neuropsychological previous testing age value](https://cde.nlm.nih.gov/deView?tinyId=h6P2tUrhYx2)

---

Data Element Name: Age of onset

Question Text: Age of onset of symptoms

Definition: 

Value Type: Number

Registration Status: Qualified

Identifier: LOINC: 65223-0

Reference URL: [Age of onset](https://cde.nlm.nih.gov/deView?tinyId=QkYmu9jhse)

---

Data Element Name: In the past 7 days My child felt accepted by other kids his/her age.

Question Text: My child felt accepted by other kids his/her age.

Definition: 

Value Type: Value List

                    Value 1: 0 
                    
                    Value 2: 1 
                    
                    Value 3: 2 
                    
                    Value 4: 3 
                    
                    Value 5: 4 

Registration Status: Qualified

Identifier: Assessment Center: Pf4socabil4

Reference URL: [In the past 7 days My child felt accepted by other kids his/her age.](https://cde.nlm.nih.gov/deView?tinyId=7JBqu1uhJ7e)

In [23]:
askQuestion("pain surveys",25)

Here are the Common Data Elements (CDEs) with a Registration Status of Qualified:

---

Data Element Name: Headache Screening Survey (HSS) - household member headache pain scale

Question Text: Headache Screening Survey (HSS) - household member headache pain scale

Definition: The scale to rate the pain caused by a headache experienced by each members of a participant's household who suffer from severe headaches from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Value List

Permissible Values: 
- Extremely severe pain 
- Severe pain 
- Moderately severe pain 
- Mild pain 

Registration Status: Qualified

Identifier: NINDS: C13494

Reference URL: [Headache Screening Survey (HSS) - household member headache pain scale](https://cde.nlm.nih.gov/deView?tinyId=7YDhF_Afs3q)

---

Data Element Name: 36-item Short Form Health Survey (SF-36) - Pain interfere work score

Question Text: 36-item Short Form Health Survey (SF-36) - Pain interfere work score

Definition: Score of bodily pain interfering with work (in home and outside) during the past 4 weeks as part of version 1.0 of 36-item Short Form Health Survey (SF-36)

Value Type: Value List

Permissible Values: 
- 1 
- 2 
- 3 
- 4 
- 5 

Registration Status: Qualified

Identifier: NINDS: C08522

Reference URL: [36-item Short Form Health Survey (SF-36) - Pain interfere work score](https://cde.nlm.nih.gov/deView?tinyId=k7yAU_K9Cfu)

---

Data Element Name: Headache Screening Survey (HSS) - household members headache affect scale

Question Text: Headache Screening Survey (HSS) - household members headache affect scale

Definition: The scale to rate the affect caused by a headache experienced by each members of a participant's household who suffer from severe headaches from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Value List

Permissible Values: 
- Able to work/function normally 
- Working ability or activity impaired to some degree 
- Working ability or activity severely impaired 
- Bed rest required 

Registration Status: Qualified

Identifier: NINDS: C13495

Reference URL: [Headache Screening Survey (HSS) - household members headache affect scale](https://cde.nlm.nih.gov/deView?tinyId=_xutYA7f8_Q)

---

Data Element Name: 36-item Short Form Health Survey (SF-36) - Bodily pain score

Question Text: 36-item Short Form Health Survey (SF-36) - Bodily pain score

Definition: Score of bodily pain during the past 4 weeks as part of version 1.0 of 36-item Short Form Health Survey (SF-36)

Value Type: Value List

Permissible Values: 
- 1 
- 2 
- 3 
- 4 
- 5 
- 6 

Registration Status: Qualified

Identifier: NINDS: C08521

Reference URL: [36-item Short Form Health Survey (SF-36) - Bodily pain score](https://cde.nlm.nih.gov/deView?tinyId=0a4z8t2uH1t)

---

Data Element Name: Headache Screening Survey (HSS) - household member headache symptom type

Question Text: Headache Screening Survey (HSS) - household member headache symptom type

Definition: The types of headache symptoms experience by each members of a participant's household who suffer from severe headaches from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Value List

Permissible Values: 
- Nausea 
- Vomiting 
- Pain on one side of head only 
- Pulsating/throbbing headaches 
- Pain-free intervals of days or weeks between severe headache attacks 
- Sensitivity to light 
- Sensitivity to noise 
- Blurring of vision 
- Seeing shimmering lights, circles, other shapes, or colors before the eyes, before the headache starts 
- Numbness of lips, tongue, fingers, or legs before the headache starts 

Registration Status: Qualified

Identifier: NINDS: C13490

Reference URL: [Headache Screening Survey (HSS) - household member headache symptom type](https://cde.nlm.nih.gov/deView?tinyId=SP5CVSCGVCo)

---

Data Element Name: 12-item Short Form Health Survey Version 2 (SF-12v2) - Bodily Pain Raw Score

Question Text: 12-item Short Form Health Survey Version 2 (SF-12v2) - Bodily Pain Raw Score

Definition: Raw score of bodily pain (BP) as part of version 2.0 of 12-item Short Form Health Survey (SF-12)

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C52353

Reference URL: [12-item Short Form Health Survey Version 2 (SF-12v2) - Bodily Pain Raw Score](https://cde.nlm.nih.gov/deView?tinyId=QkSlQO5AcQ)

---

Data Element Name: Headache Screening Survey (HSS) - household member headache unable to work scale

Question Text: Headache Screening Survey (HSS) - household member headache unable to work scale

Definition: The scale to rate the ability to function as a result of a headache experienced by each members of a participant's household who suffer from severe headaches from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Value List

Permissible Values: 
- 0 days 
- Less than 1 day 
- 1-2 days 
- 3-5 days 
- 6 or more days 

Registration Status: Qualified

Identifier: NINDS: C13496

Reference URL: [Headache Screening Survey (HSS) - household member headache unable to work scale](https://cde.nlm.nih.gov/deView?tinyId=2kH316KYTen)

---

Data Element Name: Headache Screening Survey (HSS) - household members suffering severe headache count

Question Text: Headache Screening Survey (HSS) - household members suffering severe headache count

Definition: The total number of members of participant's household who suffer from severe headaches from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C13486

Reference URL: [Headache Screening Survey (HSS) - household members suffering severe headache count](https://cde.nlm.nih.gov/deView?tinyId=Uz4pmVcTWvA)

---

Data Element Name: Headache Screening Survey (HSS) - household member headache occurrence year frequency

Question Text: Headache Screening Survey (HSS) - household member headache occurrence year frequency

Definition: The number headaches experienced by each members of a participant's household who suffer from severe headaches in a year from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Number

Registration Status: Qualified

Identifier: NINDS: C13493

Reference URL: [Headache Screening Survey (HSS) - household member headache occurrence year frequency](https://cde.nlm.nih.gov/deView?tinyId=8R6rdEAdfW9)

---

Data Element Name: PROMIS Adult Short Form 8a Pain Interference v1.0 PAININ36 6 Past 7 Days How Much Did Pain Interfere with Enjoyment of Social Activities Score 5 Point Scale

Question Text: How much did pain interfere with your enjoyment of social activities?

Definition: A person's stated observation of how much in the past 7 days pain interfered with enjoyment of social activities, using a five-point Likert scale.

Value Type: Value List

Permissible Values: 
- 1 
- 2 
- 3 
- 4 
- 5 

Registration Status: Qualified

Identifier: caDSR: 4633457

Reference URL: [PROMIS Adult Short Form 8a Pain Interference v1.0 PAININ36 6 Past 7 Days How Much Did Pain Interfere with Enjoyment of Social Activities Score 5 Point Scale](https://cde.nlm.nih.gov/deView?tinyId=Xkbg7HaX11l)

---

Data Element Name: PROMIS Adult Short Form 6b Pain Interference v1.0 PAININ14 Past 7 Days How Much Did Pain Interfere with Doing Tasks Away from Home Score 5 Point Scale

Question Text: How much did pain interfere with doing your tasks away from home (e.g., getting groceries, running errands)?

Definition: A person's stated observation of how much in the past 7 days pain interfered with doing tasks away from home such as getting groceries or running errands, using a five-point Likert scale.

Value Type: Value List

Permissible Values: 
- 1 
- 2 
- 3 
- 4 
- 5 

Registration Status: Qualified

Identifier: caDSR: 4633480

Reference URL: [PROMIS Adult Short Form 6b Pain Interference v1.0 PAININ14 Past 7 Days How Much Did Pain Interfere with Doing Tasks Away from Home Score 5 Point Scale](https://cde.nlm.nih.gov/deView?tinyId=719TLbQykg)

---

Data Element Name: PROMIS Adult Short Form 8a Pain Interference v1.0 PAININ34 4 Past 7 Days How Much Did Pain Interfere with Household Chores Score 5 Point Scale

Question Text: How much did pain interfere with your household chores?

Definition: A person's stated observation of how much in the past 7 days pain interfered with household chores, using a five-point Likert scale.

Value Type: Value List

Permissible Values: 
- 1 
- 2 
- 3 
- 4 
- 5 

Registration Status: Qualified

Identifier: caDSR: 4633458

Reference URL: [PROMIS Adult Short Form 8a Pain Interference v1.0 PAININ34 4 Past 7 Days How Much Did Pain Interfere with Household Chores Score 5 Point Scale](https://cde.nlm.nih.gov/deView?tinyId=7JeeWYeVkyg)

---

Data Element Name: PROMIS Adult Short Form 6b Pain Interference v1.0 PAININ10 Past 7 Days How Much Did Pain Interfere with Enjoyment of Recreational Activities Score 5 Point Scale

Question Text: How much did pain interfere with your enjoyment of recreational activities?

Definition: A person's stated observation of how much in the past 7 days pain interfered with their enjoyment of recreational activities, using a five-point Likert scale.

Value Type: Value List

Permissible Values: 
- 1 
- 2 
- 3 
- 4 
- 5 

Registration Status: Qualified

Identifier: caDSR: 4633479

Reference URL: [PROMIS Adult Short Form 6b Pain Interference v1.0 PAININ10 Past 7 Days How Much Did Pain Interfere with Enjoyment of Recreational Activities Score 5 Point Scale](https://cde.nlm.nih.gov/deView?tinyId=7yzaI4mk1g)

---

Data Element Name: Headache Screening Survey (HSS) - household member sex type

Question Text: Headache Screening Survey (HSS) - household member sex type

Definition: The sex of each members of a participant's household who suffer from severe headaches from Headache Screening Survey (HSS) which is a questionnaire that screens for alcohol dependency without it appearing to be the main focus of the instrument.

Value Type: Value List

Permissible Values: 
- Male 
- Female 

Registration Status: Qualified

Identifier: NINDS: C13489

Reference URL: [Headache Screening Survey (HSS) - household member sex type](https://cde.nlm.nih.gov/deView?tinyId=hfA60EMB7mf)

---

Data Element Name: 36-item Short Form Health Survey (SF-36) - Work not carefully emotional problems indicator

Question Text: 36-item Short Form Health Survey (SF-36) - Work not carefully emotional problems indicator

Definition: Indicator for surveyee-reported doing work or other activities not as carefully as usual as a result of emotional problems during the past 4 weeks as part of version 1.0 of 36-item Short Form Health Survey (SF-36)

Value Type: Value List

Permissible Values: 
- 1 
- 2 

Registration Status: Qualified

Identifier: NINDS: C08519

Reference URL: [36-item Short Form Health Survey (SF-36) - Work not carefully emotional problems indicator](https://cde.nlm.nih.gov/deView?tinyId=puJ1jYrEquQ)

---